In [101]:

from sklearn import datasets
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader


In [102]:
# 获取鸢尾花数据
iris_datas = datasets.load_iris(return_X_y=True)
target = iris_datas[1]
iris_datas=torch.tensor(iris_datas[0]).float()

print(target)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


## 自定义数据集

In [103]:
class IrisDataset(Dataset):
    def __init__(self, data,mode, target = None):
        if mode == "test":
            self.data = data
        else:
            if mode == "train":
                idx = [i for i in range(len(data)) if i % 5 !=0 ]
            elif mode == "dev":
                idx = [i for i in range(len(data)) if i % 5 ==0 ]
            self.data = data[idx]

            self.target = target[idx]
    def __getitem__(self, index):
        return self.data[index]
    def __len__(self):
        return len(self.data)

## 数据加载器


In [104]:
def pre_datalpader(dataset,batch_size):
    return DataLoader(dataset=dataset,batch_size=batch_size,shuffle=True)

## 定义神经网络


In [105]:
class IrisClassifier(nn.Module):
    def __init__(self,input_dim, n_classes):
        super(IrisClassifier, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, n_classes)
        )
        self.criterion = nn.MSELoss(reduction='mean')
    def forward(self, x):
        return self.net(x).squeeze()
    def cal_loss(self, pred, target):
        # 使用二叉熵损失函数
        loss = self.criterion(pred, target)
        return loss


## 训练神经网络

In [106]:
def train(model,dataset,config,target):
    n_epochs = config['n_epochs']
    batch_size = config['batch_size']
    learning_rate = config['learning_rate']
    epochs = 0
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    tr_load = pre_datalpader(dataset,batch_size)
    while epochs < n_epochs:
        # 训练模式
        model.train()
        running_loss = 0.0
        for x,y,*_ in tr_load:
            optimizer.zero_grad()
            x,y = x.to(config['device']), y.to(config['device'])
            pred = model(x)
            mse_loss = model.cal_loss(pred, y)
            mse_loss.backward()
            optimizer.step()
            running_loss += mse_loss.item() * x.size(0)
        epoch_loss = running_loss / len(tr_load.dataset)
        epochs += 1
        print('epoch: {}, loss: {:.4f}'.format(epochs, epoch_loss))



In [107]:
def test(model,dataset,config):
    model.eval()
    running_loss = 0.0
    correct = 0
    batch_size = config['batch_size']
    td_load = pre_datalpader(dataset,batch_size)
    with torch.no_grad():
        total = 0.0
        for x,y,*_ in td_load:
            x,y = x.to(config['device']), y.to(config['device'])
            pred = model(x)
            print(type(pred))
            loss = model.cal_loss(pred, y)
            running_loss += loss.item() * x.size(0)
            # _ ,pred = torch.max(pred, 1)
            total +=   y.size(0)
            correct = correct + (pred == y).sum().item()
    epoch_loss = running_loss / len(dataset)
    accuracy = 100.0 * correct / total
    print(f'test_loss:{epoch_loss}, accuracy: {correct}')
    return accuracy

In [108]:
config = {
    "n_epochs" : 300,
    "batch_size" : 8,
    "learning_rate" : 0.001,
    'mode' : 'train',
    'input_dim' : 4,
    'n_classes' : 4,
    'device':'cpu'
}
model = IrisClassifier(config['input_dim'],config['n_classes'])
tr_dataset = IrisDataset(iris_datas,config['mode'],target)
train(model,tr_dataset,config,target)



epoch: 1, loss: 8.2972
epoch: 2, loss: 5.6818
epoch: 3, loss: 3.2337
epoch: 4, loss: 1.9923
epoch: 5, loss: 1.0501
epoch: 6, loss: 0.8396
epoch: 7, loss: 0.6527
epoch: 8, loss: 0.9514
epoch: 9, loss: 0.7483
epoch: 10, loss: 0.8599
epoch: 11, loss: 0.6104
epoch: 12, loss: 0.5847
epoch: 13, loss: 0.6632
epoch: 14, loss: 0.6008
epoch: 15, loss: 0.5832
epoch: 16, loss: 0.7241
epoch: 17, loss: 0.5767
epoch: 18, loss: 0.6844
epoch: 19, loss: 0.7427
epoch: 20, loss: 0.5067
epoch: 21, loss: 0.4456
epoch: 22, loss: 0.7030
epoch: 23, loss: 1.0036
epoch: 24, loss: 0.5940
epoch: 25, loss: 0.8752
epoch: 26, loss: 0.5509
epoch: 27, loss: 0.7783
epoch: 28, loss: 0.8019
epoch: 29, loss: 0.6417
epoch: 30, loss: 0.3358
epoch: 31, loss: 0.5688
epoch: 32, loss: 0.3867
epoch: 33, loss: 0.5311
epoch: 34, loss: 0.6640
epoch: 35, loss: 0.9108
epoch: 36, loss: 0.5507
epoch: 37, loss: 0.6048
epoch: 38, loss: 0.6211
epoch: 39, loss: 0.5533
epoch: 40, loss: 0.6164
epoch: 41, loss: 0.5517
epoch: 42, loss: 0.5311
e

In [109]:
te_dataset = IrisDataset(iris_datas,"dev",target)
test(model,te_dataset,config)
# te_dataset.target

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
test_loss:0.4910810391108195, accuracy: 0


0.0